# Prepare Optimized Model for Deployment

# Freeze Fully Optimized Graph

In [ ]:
from tensorflow.python.tools import freeze_graph

model_parent_path = './linear_model/cpu/checkpoint'

model_graph_path = '%s/graph.pb' % model_parent_path
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
model_checkpoint_path = '%s/model.ckpt' % model_parent_path

freeze_graph.freeze_graph(input_graph=model_graph_path, 
                          input_saver="",
                          input_binary=True, 
                          input_checkpoint=model_checkpoint_path,
                          output_node_names="add",
                          restore_op_name="save/restore_all", 
                          filename_tensor_name="save/Const:0",
                          output_graph=frozen_model_graph_path, 
                          clear_devices=True, 
                          initializer_nodes="")
print(frozen_model_graph_path)

### File Size

In [ ]:
%%bash

ls -l ./linear_model/cpu/checkpoint/

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
from google.protobuf import text_format
from tensorflow.core.framework import graph_pb2

def convert_graph_to_dot(input_graph, output_dot, is_input_graph_binary):
    graph = graph_pb2.GraphDef()
    with open(input_graph, "rb") as fh:
        if is_input_graph_binary:
            graph.ParseFromString(fh.read())
        else:
            text_format.Merge(fh.read(), graph)
    with open(output_dot, "wt") as fh:
        print("digraph graphname {", file=fh)
        for node in graph.node:
            output_name = node.name
            print("  \"" + output_name + "\" [label=\"" + node.op + "\"];", file=fh)
            for input_full_name in node.input:
                parts = input_full_name.split(":")
                input_name = re.sub(r"^\^", "", parts[0])
                print("  \"" + input_name + "\" -> \"" + output_name + "\";", file=fh)
        print("}", file=fh)
        print("Created dot file '%s' for graph '%s'." % (output_dot, input_graph))
        

In [ ]:
input_graph='./linear_model/cpu/checkpoint/graph.pb'
output_dot='./frozen_model_graph_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png ./frozen_model_graph_cpu.dot \
    -o ./frozen_model_graph_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('./frozen_model_graph_cpu.png')

# Save Model for Deployment and Inference

## Reset Default Graph

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

## Create New Session

In [ ]:
sess = tf.Session()

## Load Frozen Graph

In [ ]:
from tensorflow.python.tools import inspect_checkpoint

inspect_checkpoint.print_tensors_in_checkpoint_file(file_name="./linear_model/cpu/checkpoint/model.ckpt",
                                                    tensor_name="",
                                                    all_tensors=True,
                                                    all_tensor_names=True)

In [ ]:
saver = tf.train.import_meta_graph('./linear_model/cpu/checkpoint/model.ckpt.meta')
saver.restore(sess, './linear_model/cpu/checkpoint/model.ckpt')

model_parent_path = './linear_model/cpu/checkpoint/'
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
print(frozen_model_graph_path)

with tf.gfile.GFile(frozen_model_graph_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

tf.import_graph_def(
    graph_def, 
    input_map=None, 
    return_elements=None, 
    name="", 
    op_dict=None, 
    producer_op_list=None
)

print("weights = ", sess.run("weights:0"))
print("bias = ", sess.run("bias:0"))

## Create `SignatureDef` Asset for TensorFlow Serving


In [ ]:
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils

graph = tf.get_default_graph()

x_observed = graph.get_tensor_by_name('x_observed:0')
y_pred = graph.get_tensor_by_name('add:0')

inputs_map = {'inputs': x_observed}
outputs_map = {'outputs': y_pred}

predict_signature = signature_def_utils.predict_signature_def(
                inputs = inputs_map, 
                outputs = outputs_map)
print(predict_signature)

## Save Model with Assets


In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants
from shutil import rmtree
from tensorflow.contrib import predictor
import numpy as np
import os

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'

os.makedirs(saved_model_path, exist_ok=True)
rmtree(saved_model_path)

import tensorflow.saved_model as saved_model
from shutil import rmtree

saved_model.simple_save(sess,
            saved_model_path,
            inputs={'inputs': x_observed},
            outputs={"outputs": y_pred})

In [ ]:
import os
print(saved_model_path)
os.listdir(saved_model_path)
os.listdir('%s/variables' % saved_model_path)

In [ ]:
%%bash
echo "./linear_model/cpu/pipeline_tfserving/0"
echo ""
ls -al ./linear_model/cpu/pipeline_tfserving/0

## Inspect with [Saved Model CLI](https://www.tensorflow.org/programmers_guide/saved_model_cli)
Note:  This takes a minute or two for some reason.  Please be patient.

In [ ]:
import subprocess

output = subprocess.run(["saved_model_cli", "show", \
                "--dir", saved_model_path, "--all"], \
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

print(output.stdout.decode('utf-8'))